In [1]:
import numpy as np
import os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

## 数据集读取
- Mnist数据是图像数据：(28,28,1)的灰度图

In [5]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("MNIST original")
mnist

InvalidURL: URL can't contain control characters. '/api/v1/json/data/list/data_name/mnist original/limit/2/status/active/' (found at least ' ')